In [1]:
import pandas as pd

In [2]:
filepath = '/Volumes/backup_128G/z_repository/Yumin_data/玉敏_俄羅斯課本的研究'
file_dic = 'HSK-2012.xls'
to_dic = 'HSK-2012_5000W_20190517_v1.xlsx'

read_dic = '{0}/{1}'.format(filepath, file_dic)
write_dic = '{0}/{1}'.format(filepath, to_dic)

In [3]:
def wordRow(wordCn, wordTw, level):
    return {'WordCn':wordCn, 'WordTw':wordTw, 'Level':level}

In [4]:
# 繁簡轉換器, https://pypi.org/project/hanziconv/0.2.1/
# from hanziconv import HanziConv
# HanziConv 轉得不是很好，例如：巧克力=>巧剋力
import opencc

levelMap = {'一级':1, '二级':2, '三级':3, '四级':4, '五级':5, '六级':6}
def Df2WordList(INdf):
    wordList = []
    for idx in range(0, len(INdf)):
        row = INdf.loc[idx]
        words = str(row[0]).replace('）', '').split('（')
        
#         word = HanziConv.toTraditional(words[0])
        word = opencc.convert(words[0], config='s2tw.json')
        level = 9
        if len(words) == 2:
            if words[1] in levelMap:
                level = levelMap[words[1]]
            else:
                print("WRONG LEVEL 1:", row)
                continue
        elif len(words) == 3:
            if words[2] in levelMap:
                level = levelMap[words[2]]
            else:
                print("WRONG LEVEL 2:", row)
                continue
        else:
            print("WRONG LEVEL 3:", row)
            continue

        if idx % 100 == 0:
            print("parsed:", idx)

        wordList.append(wordRow(words[0], word, level))
        
    return wordList

In [5]:
def runParsing(wordList):
    newWordList = []
    for theWord in wordList:
        word = theWord[0]
        level = theWord[1]

        if '/' in word:
            wordArr = word.split('/')
            for sword in wordArr:
                if sword != '':
                    newWordList.append(wordRow(sword, level))
            continue

        if '(' in word:
            wordArr = word.split('(')

            for widx in range(0, len(wordArr)):
                sword = wordArr[widx].replace(')', '').replace(' ', '')
                if widx == 0:
                    newWordList.append(wordRow(sword, level))
                elif sword.isalpha() == True:
                    oriWord = wordArr[0][0:len(wordArr[0])-1]
                    if sword.startswith(oriWord):
                        newWordList.append(wordRow(sword, level))
                    else:
                        completeWord = '{0}{1}'.format(wordArr[0], sword)
                        newWordList.append(wordRow(completeWord, level))

            continue

        newWordList.append(wordRow(word, level))
    
    return newWordList

In [6]:
# sheet2Level = {'準備級一級':1,'準備級二級':2,'入門級':3,'基礎級':4,'進階級':5,'高階級':6,'流利級':7}
# sheet2Level = {'準備級一級':1}
with pd.ExcelFile(read_dic) as reader:
    with pd.ExcelWriter(write_dic) as writer:
        dicDf = pd.read_excel(reader, "新HSK5000词", header=None)
#         print(dicDf)

        wordList = Df2WordList(dicDf)

        outDf = pd.DataFrame(wordList)
        outDf.to_excel(writer, "HSK5000", index=False)
        
        writer.save()

parsed: 0
parsed: 100
parsed: 200
parsed: 300
parsed: 400
parsed: 500
parsed: 600
parsed: 700
parsed: 800
parsed: 900
parsed: 1000
parsed: 1100
parsed: 1200
parsed: 1300
parsed: 1400
parsed: 1500
parsed: 1600
parsed: 1700
parsed: 1800
parsed: 1900
parsed: 2000
parsed: 2100
parsed: 2200
parsed: 2300
parsed: 2400
parsed: 2500
parsed: 2600
parsed: 2700
parsed: 2800
parsed: 2900
parsed: 3000
parsed: 3100
parsed: 3200
parsed: 3300
parsed: 3400
parsed: 3500
parsed: 3600
parsed: 3700
parsed: 3800
parsed: 3900
parsed: 4000
parsed: 4100
parsed: 4200
parsed: 4300
parsed: 4400
parsed: 4500
parsed: 4600
parsed: 4700
parsed: 4800
parsed: 4900
